In [1]:
## 종훈형 피드백
### Vasopressor를 전처리 할 때 줬냐 안줬냐를 볼 때에는 상관이 없으나, 기준은 Norepinephrine이 되어야함. Vasopressor는 주는 순서가 있음. 
### Norepinephrine(부작용 덜 함) -> Epinephrine -> Dobutamin, Vassopressin(에피네프린도 들지 않는다면 굉장히 심한 혈액 순환이 안될 때 주는 것임), Dopamine 
### Phenylephrine (일반적으로 외과 수술 이후에 혹시 몰라서 주는 굉장히 약한 승압제) 
### 따라서 같은 용량을 주더라도 Vassopressin과 Phenylephrine 사이에선 Vassopressin이 더 강한 약물.
### 단위 통일을 먼저 하고 약의 효과를 그 뒤에 곱해주는 것임.
### Vasopressor는 amount보다 rate를 좀 더 중요하게 보는 경향이 있음.

### Fluid도 Vasopressor와 마찬가지로 단위 변환이 필요함. 얘는 Amount가 좀 더 중요하긴 하나 rate. rate을 보고 이름에 Bolus가 붙어 있진 않더라도 이와 마찬가지인 값들을 확인해 보는 것을 추천.

### Urine Output -> Input이 있는데 Output이 0인 경우 이를 이월해서 몸에 남아있다는 정보를 추가.

### 의료 도메인 관련 의대 논문을 추려서 읽으면 Domain Knowledge를 늘리는 것을 추천.

### vital sign 중에선 Non-Invasive를 먼저 기준으로 진행하고 만약 겹치는 구간이 있다면 Invasive 값으로 대체. 이는 Disatolic과 Systolic 2개 모두에 존재해야함. 왜냐하면 측정 방법이 다르면 서로 값이 달라지기 때문에 더 부정확한 결과를 초래할 수 있음.

### 가능한 데이터를 다 가져와서 처리

#### 가장 핵심은 단위가 같은지 확인하면서 진행하기. Demographic 정보 중에선 단위 환산에 사용할 땐 Wegiht 대신 Ideal Weight를 사용하는 것을 추천 -> 왜냐하면 단위 환산 중에 kg/h 이러한 단위가 있는데 이를 적용시킬 때에는 정상적인 체중을 기준으로 계산해야하기 때문임. 

### Invasive는 전기 신호로 인해 flash 현상이 발생할 수 있는데 이 데이터는 완벽한 이상치임. 제거하고 진행해야함. --> flash_condition1 = (data['IBlood pressure systolic']*0.9 <= data['IBlood pressure diastolic'])

### 연구 성격에 따라서 중요한 변수들에 대해선 좀 더 민감하게 전처리 해도 무방함.

### 처음에 진행할 때 환자를 대략 100명 정도 Sampling을 먼저 진행하고 검정하면서 진행해보기 

### Demographic -> vital과 lab value를 먼저 전처리 -> fluids -> vasopressor(보통 rate으로 진행) 

# 0) Prepare base dataframes and select usage variables
    - It cotains import tools, fix seed, load datasets, feature selection for only using interesting variables, etc.. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import random 
import os 
import gc
import math
import json
import optuna

from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

from tqdm import tqdm

c:\Users\korea\miniconda3\envs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def fix_seed(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 

my_seed = 42
fix_seed(my_seed)

In [3]:
with open('./utils/concept-dict.json', 'r') as f:
    concept_dict = json.load(f)

static_vars = ["age", "sex", "height", "weight"]

dynamic_vars = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir",
                  "bnd", "bun", "ca", "cai", "ck", "ckmb", "cl", "crea", "crp", 
                  "dbp", "fgn", "fio2", "glu", "hgb", "hr", "inr_pt", "k", "lact",
                  "lymph", "map", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", 
                  "o2sat", "pco2", "ph", "phos", "plt", "po2", "ptt", "resp", "sbp", 
                  "temp", "tnt", "urine", "wbc"]

# concept_dict는 다음과 같은 구조로 되어있음 

"""
concept_dict 구조도 

변수 이름 > unit, min, max, category, sources 

- min : 이상치 처리 시 사용한 min값
- max : 이상치 처리 시 사용한 max 값
- cateogry: 어떤 종류의 변수인지 

-- sources의 정보들
ids : 해당 변수 고유 식별 번호
table : 어떤 테이블에 존재하는지?
sub_var : 식별 번호가 table의 어떤 column에서 조회해야 하는지?
val_var : itemid가 없는 경우 그냥 이 colmn 자체의 값들로 가져오면 됨
call_back : 어떠한 전처리를 고려했는지? (e.g.) 단위 변환등 어떤 전처리를 수행해야 하는지 서술해놓음
"""

# e.g.
concept_dict[dynamic_vars[16]]['sources']['miiv']

[{'ids': 50889,
  'table': 'labevents',
  'sub_var': 'itemid',
  'callback': "convert_unit(binary_op(`*`, 10), 'mg/L', 'mg/dl')"}]

In [4]:
# 해당 데이터셋마다 필요한 값들을 excel 형식으로 변환해보자.

def select_vars(db_name, var_list, base_dict):
    """
    min, max => 이상치 처리 시 사용
    cateogries => 해당 변수 category 분류 시 사용
    source_table => 어느 테이블에서 가져와야 하는지
    source_columns => 해당 테이블 어느 컬럼에서 가져와야 하는지
    source_itemid => 해당 테이블의 컬럼에서 어떤 id로 존재하는지
    """
    var_normal_min = []
    var_normal_max = []
    var_unit = []
    source_category = []
    source_table = []
    source_column = []
    source_itemid = []
    source_callback = []
    source_vars = [] # to map other lists
    for var in tqdm(var_list):

        for i in range(len(base_dict[var]['sources'][db_name])): # 여러 테이블에 분산되어 있는 경우 길이가 2가 넘을 수 있기 때문

            source_table.append(base_dict[var]['sources'][db_name][i]['table'])

            try:
                source_column.append(base_dict[var]['sources'][db_name][i]['sub_var'])
            except:
                source_column.append(base_dict[var]['sources'][db_name][i]['val_var'])

            try:
                if [223761, 224027] == base_dict[var]['sources'][db_name][i]['ids']: # temp 관련 변수에서 skin temp는 제외 
                    source_itemid.append(223761)
                
                else:
                    source_itemid.append(base_dict[var]['sources'][db_name][i]['ids'])
            except:
                source_itemid.append(None)

            try:
                source_callback.append(base_dict[var]['sources'][db_name][i]['callback'])
            except:
                source_callback.append(None)


            source_category.append(base_dict[var]['category'])

            try:
                var_unit.append(base_dict[var]['unit'])
            except:
                var_unit.append(None)

            try:
                var_normal_min.append(base_dict[var]['min'])
            except:
                var_normal_min.append(None)
            
            try:
                var_normal_max.append(base_dict[var]['max'])
            except:
                var_normal_max.append(None)

            source_vars.append(var)
        
    return var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars

var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars  = select_vars('miiv', static_vars + dynamic_vars, concept_dict)

MAPPING_DF = pd.DataFrame({
    'var_name' : source_vars,
    'normal_min' : var_normal_min,
    'normal_max' : var_normal_max,
    'category' : source_category,
    'table' : source_table,
    'column' : source_column,
    'itemid' : source_itemid,
    'unit' : var_unit,
    'method' : source_callback 
})

MAPPING_DF

100%|██████████| 52/52 [00:00<?, ?it/s]


,var_name,normal_min,normal_max,category,table,column,itemid,unit,method
0,age,0.0,100.0,demographics,patients,anchor_age,None,years,None
1,sex,NaN,NaN,demographics,patients,gender,None,None,"apply_map(c(M = 'Male', F = 'Female'))"
2,height,10.0,230.0,demographics,chartevents,itemid,226707,cm,"convert_unit(binary_op(`*`, 2.54), 'cm', '^in')"
3,weight,1.0,500.0,demographics,chartevents,itemid,226512,kg,None
4,alb,0.0,6.0,chemistry,labevents,itemid,50862,g/dL,None
5,alp,0.0,NaN,chemistry,labevents,itemid,50863,"[IU/L, U/l]",None
6,alt,0.0,NaN,chemistry,labevents,itemid,50861,"[IU/L, U/l]",None
7,ast,0.0,NaN,chemistry,labevents,itemid,50878,"[IU/L, U/l]",None
8,be,-25.0,25.0,blood gas,labevents,itemid,50802,"[mEq/L, mmol/l]",None
9,bicar,5.0,50.0,chemistry,labevents,itemid,50882,"[mEq/L, mmol/l]",None


In [ ]:
# # 해당 데이터셋마다 필요한 값들을 excel 형식으로 변환해보자.

# def select_vars(db_name, var_list, base_dict):
#     """
#     min, max => 이상치 처리 시 사용
#     cateogries => 해당 변수 category 분류 시 사용
#     source_table => 어느 테이블에서 가져와야 하는지
#     source_columns => 해당 테이블 어느 컬럼에서 가져와야 하는지
#     source_itemid => 해당 테이블의 컬럼에서 어떤 id로 존재하는지
#     """
#     var_normal_min = []
#     var_normal_max = []
#     var_unit = []
#     source_category = []
#     source_table = []
#     source_column = []
#     source_itemid = []
#     source_callback = []
#     source_regex = []
#     source_vars = [] # to map other lists
#     for var in tqdm(var_list):

#         for i in range(len(base_dict[var]['sources'][db_name])): # 여러 테이블에 분산되어 있는 경우 길이가 2가 넘을 수 있기 때문

#             source_table.append(base_dict[var]['sources'][db_name][i]['table'])

#             try:
#                 source_column.append(base_dict[var]['sources'][db_name][i]['sub_var'])
#             except:
#                 source_column.append(base_dict[var]['sources'][db_name][i]['val_var'])

#             try:
#                 source_itemid.append(base_dict[var]['sources'][db_name][i]['ids'])
#             except:
#                 source_itemid.append(None)

#             try:
#                 source_callback.append(base_dict[var]['sources'][db_name][i]['callback'])
#             except:
#                 source_callback.append(None)
            
#             try:
#                 source_regex.append(base_dict[var]['sources'][db_name][i]['regex'])
#             except:
#                 source_regex.append(None)

#             source_category.append(base_dict[var]['category'])

#             try:
#                 var_unit.append(base_dict[var]['unit'])
#             except:
#                 var_unit.append(None)

#             try:
#                 var_normal_min.append(base_dict[var]['min'])
#             except:
#                 var_normal_min.append(None)
            
#             try:
#                 var_normal_max.append(base_dict[var]['max'])
#             except:
#                 var_normal_max.append(None)


#             source_vars.append(var)
        
#     return var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex

# var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex  = select_vars('eicu', static_vars + dynamic_vars, concept_dict)

# MAPPING_DF = pd.DataFrame({
#     'var_name' : source_vars,
#     'normal_min' : var_normal_min,
#     'normal_max' : var_normal_max,
#     'category' : source_category,
#     'table' : source_table,
#     'column' : source_column,
#     'itemid' : source_itemid,
#     'unit' : var_unit,
#     'method' : source_callback,
#     'regex' : source_regex
# })

# MAPPING_DF

In [5]:
# Original Data Load..
ROOT_DIR = '/Users/korea/datasets/physionet.org/files/mimiciv/3.0'
ROOT_DIR = Path(ROOT_DIR)

# Make Output Folder
if not os.path.exists(ROOT_DIR/'preprocessed_yaib'):
	os.makedirs(ROOT_DIR/'preprocessed_yaib')

## ICU
# care_giver_origin = pd.read_csv(ROOT_DIR/'icu'/'caregiver.csv.gz',compression = 'gzip')
chartevents_origin = pd.read_csv(ROOT_DIR/'icu'/'chartevents.csv.gz',compression = 'gzip')
d_items_origin = pd.read_csv(ROOT_DIR/'icu'/'d_items.csv.gz',compression = 'gzip')
icustays_origin = pd.read_csv(ROOT_DIR/'icu'/'icustays.csv.gz',compression = 'gzip')
# ingredientevents_origin = pd.read_csv(ROOT_DIR/'icu'/'ingredientevents.csv.gz',compression = 'gzip')
# inputevents_origin = pd.read_csv(ROOT_DIR/'icu'/'inputevents.csv.gz',compression = 'gzip')
outputevents_origin = pd.read_csv(ROOT_DIR/'icu'/'outputevents.csv.gz',compression = 'gzip')
# procedureevents_origin = pd.read_csv(ROOT_DIR/'icu'/'procedureevents.csv.gz',compression = 'gzip')
# datetimeevents_origin = pd.read_csv(ROOT_DIR/'icu'/'datetimeevents.csv.gz',compression = 'gzip')

In [6]:
## Hosp
# drgcodes_origin = pd.read_csv(ROOT_DIR/'hosp'/'drgcodes.csv.gz',compression = 'gzip')
# d_icd_procedures_origin = pd.read_csv(ROOT_DIR/'hosp'/'d_icd_procedures.csv.gz',compression = 'gzip')
# procedures_icd = pd.read_csv(ROOT_DIR/'hosp'/'procedures_icd.csv.gz',compression = 'gzip')
# prescriptions_origin = pd.read_csv(ROOT_DIR/'hosp'/'prescriptions.csv.gz',compression = 'gzip')
patients_origin = pd.read_csv(ROOT_DIR/'hosp'/'patients.csv.gz',compression = 'gzip')
admissions_origin = pd.read_csv(ROOT_DIR/'hosp'/'admissions.csv.gz',compression = 'gzip')
d_labitems = pd.read_csv(ROOT_DIR/'hosp'/'d_labitems.csv.gz',compression = 'gzip')
# diagnoses_icd = pd.read_csv(ROOT_DIR/'hosp'/'diagnoses_icd.csv.gz',compression = 'gzip')
# d_icd_diagnoses = pd.read_csv(ROOT_DIR/'hosp'/'d_icd_diagnoses.csv.gz',compression = 'gzip')
labevents = pd.read_csv(ROOT_DIR/'hosp'/'labevents.csv.gz',compression = 'gzip')
# services_origin = pd.read_csv(ROOT_DIR/'hosp'/'services.csv.gz',compression = 'gzip')
# omr_origin = pd.read_csv(ROOT_DIR/'hosp'/'omr.csv.gz',compression = 'gzip')
# microbiology = pd.read_csv(ROOT_DIR/'hosp'/'microbiologyevents.csv.gz' , compression='gzip')

## Check for used variables 

In [8]:
# def find_interest_icdcodes(x, df, outputtype = 'icdcode'):
    
#     if outputtype == "icd_code":
#         print(df[df['long_title'].str.contains(x, case = False)][outputtype].values)
    
#     elif outputtype == "abbreviation":
#         print(df[df['long_title'].str.contains(x, case = False)][outputtype].values)
    
    
#     display(df[df['long_title'].str.contains(x, case = False)])
    
    
# find_interest_icdcodes('Acute Kidney', d_icd_diagnoses, 'icd_code')

In [ ]:
# # 관심있는 변수들을 먼저 찾기 (ICU)
# def find_interest_labels(x, df, outputtype = 'label'):
    
#     if outputtype == "label":
#         print(df[df['label'].str.contains(x, case = False, regex = False)][outputtype].values)
    
#     elif outputtype == "abbreviation":
#         print(df[df['label'].str.contains(x, case = False, regex = False)][outputtype].values)
    

#     print(df[df['label'].str.contains(x, case = False, regex = False)]['itemid'].values)
    
#     display(df[df['label'].str.contains(x, case = False, regex = False)])


# # ICU    
# find_interest_labels('platelet', d_items_origin, 'label')

# # Lab value
# find_interest_labels('platelet', d_labitems.fillna('Not Preserve'), 'label')

# itemid로 변수들 먼저 찾기 (ICU)
def find_interest_labels(x, df, outputtype = 'label'):
    
    if outputtype == "label":
        print(df[df['itemid'].isin(x)][outputtype].values)
    
    elif outputtype == "abbreviation":
        print(df[df['itemid'].isin(x)][outputtype].values)
    
    output = df[df['itemid'].isin(x)].shape[0]
    return output

# 설정한 itemid들이 모두 있는지 확인
check = []
count = 0
for iid in MAPPING_DF.itemid:
    if iid:
        if not isinstance(iid, list):
            iid = [iid]
        
        output1 = find_interest_labels(iid, d_items_origin)
        output2 = find_interest_labels(iid, d_labitems)

        output = output1 + output2
        check.append(output == len(iid))
        
        count += 1

print(len(check), count)
print(len(check) == count)

['Height']
[]
['Admission Weight (Kg)']
[]
[]
['Albumin']
[]
['Alkaline Phosphatase']
[]
['Alanine Aminotransferase (ALT)']
[]
['Asparate Aminotransferase (AST)']
[]
['Base Excess']
[]
['Bicarbonate']
[]
['Bilirubin, Total']
[]
['Bilirubin, Direct']
[]
['Bands']
[]
['Urea Nitrogen']
[]
['Calcium, Total']
[]
['Free Calcium']
[]
['Creatine Kinase (CK)']
[]
['Creatine Kinase, MB Isoenzyme']
[]
['Chloride']
[]
['Creatinine']
[]
['C-Reactive Protein']
['Arterial Blood Pressure diastolic'
 'Non Invasive Blood Pressure diastolic']
[]
[]
['Fibrinogen, Functional']
['Inspired O2 Fraction']
[]
[]
['Oxygen']
[]
['Glucose' 'Glucose']
[]
['Hemoglobin']
['Heart Rate']
[]
[]
['INR(PT)']
[]
['Potassium']
[]
['Lactate']
[]
['Lymphocytes']
['Arterial Blood Pressure mean' 'Non Invasive Blood Pressure mean'
 'ART BP Mean']
[]
[]
['MCH']
[]
['MCHC']
[]
['MCV']
[]
['Methemoglobin']
[]
['Magnesium']
[]
['Sodium']
[]
['Neutrophils']
['O2 saturation pulseoxymetry' 'SpO2 Desat Limit']
['Oxygen Saturation']
[]
[

## Select only used variables in dataframe

In [7]:
# Merge with itemid for be used data.
## ICU
chartevents_df = pd.merge(chartevents_origin, d_items_origin, how = 'left', left_on = 'itemid', right_on = 'itemid')
del chartevents_origin # Free up for memory

outputevents_df = pd.merge(outputevents_origin, d_items_origin, how = 'left', left_on = 'itemid', right_on = 'itemid')
del outputevents_origin

# procedureevents_df = pd.merge(procedureevents_origin, d_items_origin, how = 'left', left_on = 'itemid', right_on = 'itemid')
# del procedureevents_origin

icustays_df = pd.merge(icustays_origin, patients_origin, how = 'left', on = 'subject_id')
del icustays_origin


## Hosp
labevents_df = pd.merge(labevents, d_labitems, how = 'left', on = 'itemid')
del labevents

admissions_df = pd.merge(admissions_origin, patients_origin, how='left', on='subject_id')
del admissions_origin

# diagnoses_icd_df = pd.merge(diagnoses_icd, d_icd_diagnoses, how='left', on=['icd_code', 'icd_version'])
# del diagnoses_icd

# procedures_icd_df = pd.merge(procedures_icd, d_icd_procedures_origin, how = 'left', on=['icd_code', 'icd_version'])
# del procedures_icd

gc.collect()

0

In [8]:
# 매핑 규칙 
"""
1. table > column > itemid 순서대로 매칭 시작 
2. 만약 column이 itemid가 아니면 해당 column 자체의 값을 활용 
3. method가 none이 아니면 단위 변환이 필요
"""

# 사용할 ITEMID LIST 정리
ITEMID_LIST = []
for iid in MAPPING_DF.itemid:
    if iid:
        if not isinstance(iid, list):
            iid = [iid]

        ITEMID_LIST += iid

print(ITEMID_LIST)

## ICU
chartevents_df = chartevents_df[chartevents_df['itemid'].isin(ITEMID_LIST)]
outputevents_df = outputevents_df[outputevents_df['itemid'].isin(ITEMID_LIST)]

## Hosp
labevents_df = labevents_df[labevents_df['itemid'].isin(ITEMID_LIST)]

gc.collect()

# 각 itemid마다 어떤 var_name으로 가야하는지 매핑
chartevents_df = pd.merge(chartevents_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', on = 'itemid')
outputevents_df = pd.merge(outputevents_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', on = 'itemid')
labevents_df = pd.merge(labevents_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', on = 'itemid')

gc.collect()

[226707, 226512, 50862, 50863, 50861, 50878, 50802, 50882, 50885, 50883, 51144, 51006, 50893, 50808, 50910, 50911, 50902, 50912, 50889, 220051, 220180, 51214, 223835, 50816, 50809, 50931, 51222, 220045, 51237, 50971, 50813, 51244, 220052, 220181, 225312, 51248, 51249, 51250, 50814, 50960, 50983, 51256, 220277, 226253, 50817, 50818, 50820, 50970, 51265, 50821, 51275, 220210, 224688, 224689, 224690, 220050, 220179, 223762, 223761, 51003, 226557, 226558, 226559, 226560, 226561, 226563, 226564, 226565, 226566, 226567, 226584, 227510, 51301]


0

# 1) Exclusion Criteria & Checkout Valid Lab events & Outlier & Add time_since_ICU columns

## 1-1) Exclusion Criteria & Checkout valid lab & Delete exlcusion criteria `stay_id` in dataframe

1. `inunit`에서 사망한 경우 `outtime`을 `deathtime`으로 수정.
2. `los` >= 30 hours
3. `age` >= 18 & age <= 89
4. ~~`labevent`의 경우 추가적으로 `intime` **1일 전** 까지의 데이터까지 사용하되 `outtime` 이전까지의 값을 타당한 것으로 포함.~~ => 입원 기간 내의 데이터로만 변경
5. `charttime`으로 기록된 것은 `intime`, `outtime` 사이에 있는 값만 & `starttime`과 `endtime`으로 기록된 것은 `starttime`과 `endtime` 모두 `intime` 이전에 있는 것만을 제외.
    
~순서 : 1,2,3 --> 4 --> 5  

In [9]:
# Prepare ICU Stay Dataset and calculate age at intime year
## 환자의 ICU Stay 입원 기록과 Anchor year를 고려해야함.
icustays_df['intime_year'] = pd.to_datetime(icustays_df['intime']).dt.year.astype(int)

# 만약 환자가 outtime보다 더 빨리 사망한 경우 outtime을 대체하고 해당 los를 다시 계산.
def reassign_outtime_los(icustay_df, admission_df):
    df = pd.merge(icustay_df, admission_df[['subject_id', 'hadm_id', 'deathtime']], how = 'left', on = ['subject_id', 'hadm_id'])
    
    df['deathtime'] = pd.to_datetime(df['deathtime'])
    df['outtime'] = pd.to_datetime(df['outtime'])
    df['intime'] = pd.to_datetime(df['intime'])
    
    check_row = df['deathtime'] <= df['outtime']
    df.loc[check_row, 'outtime'] = df.loc[check_row, 'deathtime']
    
    df.loc[check_row, 'los'] = (df.loc[check_row, 'outtime'] - df.loc[check_row, 'intime']).dt.total_seconds() / (60*60*24)
    
    df = df.drop(columns = 'deathtime')
    
    return df
    
    
# intime만을 고려한 age -> 그러나, 본 연구에서는 yaib의 전처리와 맞추기 위해 hospital admission age인 anchor_age를 사용
def cal_intime_age(values):
    anchor_age, anchor_year, intime_year = values
    now_age = anchor_age + (intime_year - anchor_year)
    return now_age

def exclusion_criteria(df, min_los, min_age, max_age, var_age):
    
    # 1) Invalid LoS -> Maybe Zero
    df = df[df['los']>= 0]
    
    # 2) Min Los
    df = df[df['los']>= min_los]
    
    # 3) Min age : 18 & Max age <= 89
    df = df[(df[var_age] >= min_age) & (df[var_age] <= max_age)]
    
    return df.reset_index(drop = True)

icustays_df = reassign_outtime_los(icustays_df, admissions_df) # reassign outtime and los
# icustays_df['intime_age'] = icustays_df[['anchor_age','anchor_year','intime_year']].apply(cal_intime_age,axis = 1)

ex_icustays_df = exclusion_criteria(icustays_df, 1.25, 18, 89, var_age = 'anchor_age')

print(f'{icustays_df.shape[0] - ex_icustays_df.shape[0]}명의 환자 삭제 최종 stay_id 개수 {ex_icustays_df.stay_id.nunique()} 명')

31820명의 환자 삭제 최종 stay_id 개수 62638 명


In [ ]:
# lab의 hadm_id에 stay_id를 추가하는 코드.
def assign_stayid_to_lab(lab_event, ex_icustays_df, valid_days = 1):
    valid_lab = lab_event[lab_event['hadm_id'].notna()].query('hadm_id.isin(@ex_icustays_df.hadm_id.unique())')

    new_lab_rows = []
    
    valid_lab['charttime'] = pd.to_datetime(valid_lab['charttime'], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['intime'] = pd.to_datetime(ex_icustays_df['intime'], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['outtime'] = pd.to_datetime(ex_icustays_df['outtime'], format="%Y-%m-%d %H:%M:%S")

    valid_lab['stay_id'] = np.nan

    for _, row in tqdm(valid_lab.iterrows(), total = valid_lab.shape[0], desc = 'assining_stay_id'):
        standard_icu = ex_icustays_df[ex_icustays_df['hadm_id'] == row.hadm_id].sort_values('intime')
            
        for _, stay in standard_icu.iterrows():
            
            if stay['intime'] - pd.Timedelta(days=valid_days) <= row['charttime'] <= stay['outtime']: # 몇 일 전 데이터까지 사용할 것인지?
                row['stay_id'] = stay['stay_id']
                new_lab_rows.append(row)
                break
            else:
                continue

    result = pd.DataFrame(new_lab_rows)
    
    return result

valid_labevents = assign_stayid_to_lab(labevents_df, ex_icustays_df, 0) # 중환자실 입원 기간 내의 데이터만 사용.
# del labevents_df # Free up for memory

valid_labevents['unit'] = valid_labevents['unit'].astype('string')
valid_labevents.to_parquet(ROOT_DIR/'preprocessed_yaib'/f'valid_labevents.parquet', index=False)

In [ ]:
# Load Valid Labevents
valid_labevents = pd.read_parquet(ROOT_DIR/'preprocessed_yaib'/'valid_labevents.parquet', engine = 'fastparquet')
labevents_df = labevents_df[labevents_df['hadm_id'].isin(valid_labevents['hadm_id'].unique())].reset_index(drop = True)
# del labevents_df

In [10]:
def apply_exclusion_criteria(df, ex_icustays_df, time_col_name : str = 'charttime'):
    """
    Exclusion Criteria에 맞춰 invalid한 데이터 제거.
    """
    
    ## 먼저 intime, outtime의 type을 object -> datetime으로 변경.
    df[time_col_name] = pd.to_datetime(df[time_col_name], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['intime'] = pd.to_datetime(ex_icustays_df['intime'], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['outtime'] = pd.to_datetime(ex_icustays_df['outtime'], format="%Y-%m-%d %H:%M:%S")
    
    ## icustays에서 사용할 column만 추리기
    standard_merge_cols = ['subject_id', 'hadm_id', 'stay_id']
    standard_icu_cols = ['subject_id','hadm_id','stay_id','intime','outtime']
    
    merged_df = pd.merge(df, ex_icustays_df[standard_icu_cols], how = 'inner', on = standard_merge_cols) # inner로 겹치는 것들만 추리기.
    
    # charttime을 사용하는 경우
    if time_col_name == "charttime":
        
        merged_df = merged_df[(merged_df['intime'] <= merged_df[time_col_name]) & (merged_df[time_col_name] <= merged_df['outtime'])] # charttime이 intime과 outtime 사이에 있는 값만 사용. 이외 제외.
        
    # charttime이 아닌 startime, endtime을 사용하는 경우   
    else: 
        merged_df['endtime'] = pd.to_datetime(merged_df['endtime'], format="%Y-%m-%d %H:%M:%S")
        
        merged_df = merged_df[~((merged_df[time_col_name] < merged_df['intime']) & (merged_df['endtime'] <= merged_df['intime']))] # strattime, endtime이 intime보다 이전에 있는 경우 제외.
        
    print(f'Result {df.shape[0]} ---> {merged_df.shape[0]}, {df.shape[0] - merged_df.shape[0]} Delete!')
        
    return merged_df.drop(columns = ['intime', 'outtime'])

# Delete exclusion criterion stay_id in dataframe.
# labevent는 이미 위에서 처리함
chartevents_df = apply_exclusion_criteria(chartevents_df, ex_icustays_df, 'charttime')
outputevents_df = apply_exclusion_criteria(outputevents_df, ex_icustays_df, 'charttime')

Result 58276857 ---> 52446812, 5830045 Delete!
Result 4251397 ---> 3875976, 375421 Delete!


## 1-2) Unit Conversion

In [180]:
# 단위 변경해야할 것은 height, temp 뿐 / crp는 이미 mg/L로 맞춰져 있음.
display(MAPPING_DF[MAPPING_DF['method'].notna()])

display(chartevents_df[['var_name', 'itemid', 'unitname']].drop_duplicates(), MAPPING_DF[MAPPING_DF['table'] == 'chartevents'][['var_name', 'unit']])

display(valid_labevents[['var_name', 'itemid', 'valueuom', 'unit']].drop_duplicates())

,var_name,normal_min,normal_max,category,table,column,itemid,unit,method
1,sex,NaN,NaN,demographics,patients,gender,None,None,"apply_map(c(M = 'Male', F = 'Female'))"
2,height,10.0,230.0,demographics,chartevents,itemid,226707,cm,"convert_unit(binary_op(`*`, 2.54), 'cm', '^in')"
20,crp,0.0,NaN,chemistry,labevents,itemid,50889,mg/L,"convert_unit(binary_op(`*`, 10), 'mg/L', 'mg/dl')"
50,temp,32.0,42.0,vitals,chartevents,itemid,223761,"[C, °C]","convert_unit(fahr_to_cels, 'C', 'f')"


,var_name,itemid,unitname
0,fio2,223835,NaN
2,hr,220045,bpm
3,sbp,220179,mmHg
4,dbp,220180,mmHg
5,map,220181,mmHg
6,resp,220210,insp/min
8,o2sat,226253,%
9,o2sat,220277,%
10,temp,223761,°F
829,temp,223762,°C


,var_name,unit
2,height,cm
3,weight,kg
21,dbp,"[mmHg, mm Hg]"
23,fio2,%
27,hr,"[bpm, /min]"
32,map,"[mmHg, mm Hg]"
40,o2sat,"[%, % Sat.]"
47,resp,"[insp/min, /min]"
48,sbp,"[mmHg, mm Hg]"
49,temp,"[C, °C]"


,var_name,itemid,valueuom,unit
0,hgb,51222,g/dL,g/dL
1,mch,51248,pg,pg
2,mchc,51249,%,%
3,mcv,51250,fL,fL
4,plt,51265,K/uL,"['K/uL', 'G/l']"
5,wbc,51301,K/uL,"['K/uL', 'G/l']"
6,bicar,50882,mEq/L,"['mEq/L', 'mmol/l']"
7,ca,50893,mg/dL,mg/dL
8,cl,50902,mEq/L,"['mEq/L', 'mmol/l']"
9,ck,50910,IU/L,"['IU/L', 'U/l']"


### 1-2-1) Vital Signs and Lab values

In [183]:
def fahrenheit_to_celsius(x):
    return (x-32) * 5 / 9

def inch_to_cm(x):
    return x * 2.54


In [184]:
# 화씨 -> 섭씨로 변경
chartevents_df.loc[chartevents_df['itemid'] == 223761,'valuenum'] = chartevents_df[chartevents_df['itemid'] == 223761]['valuenum'].apply(fahrenheit_to_celsius)

# inch -> cm 단위 변경
chartevents_df.loc[chartevents_df['itemid'] == 226707,'valuenum'] = chartevents_df[chartevents_df['itemid'] == 226707]['valuenum'].apply(inch_to_cm)

gc.collect()

17

## 1-3) Preprocess Outlier

In [185]:
# Outlier Removal by min, max values

def filter_outliers(df, value_col):
    """
    연속형 변수 처리하기 위한 함수 
    """
    def _filtering(row):
        if not np.isnan(row['normal_min']) and not np.isnan(row['normal_max']):
            return row['normal_min'] <= row[value_col] <= row['normal_max'] 

        elif not np.isnan(row['normal_min']) and np.isnan(row['normal_max']): # min값만 있을 때
            return row['normal_min'] <= row[value_col]

        elif np.isnan(row['normal_min']) and not np.isnan(row['normal_max']): # max값만 있을 때 
            return row[value_col] <= row['normal_max'] 

        else:
            return True

    df_filtered = df.copy()
    outlier_binary = df.apply(_filtering, axis = 1)
    df_filtered['out_bin'] = outlier_binary

    df_filtered = df_filtered[df_filtered['out_bin']].reset_index(drop = True)

    return df_filtered

chartevents_df = filter_outliers(chartevents_df, 'valuenum')
valid_labevents = filter_outliers(valid_labevents, 'valuenum')
outputevents_df = filter_outliers(outputevents_df, 'value')

# 2) Disease Annotation


## 2-1) AKI Annotation

- AKI Labeling은 입원 이전 7일 이내의 baseline을 구해야하기 때문에 original labevent dataframe으로 수행. 

### 2-1-1) Preparing Variables

In [291]:
# labevent에서는 creatinine을 사용
def assign_stayid_to_lab(lab_event, ex_icustays_df, valid_days = 1):
    valid_lab = lab_event[lab_event['hadm_id'].notna()].query('hadm_id.isin(@ex_icustays_df.hadm_id.unique())')

    new_lab_rows = []
    
    valid_lab['charttime'] = pd.to_datetime(valid_lab['charttime'], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['intime'] = pd.to_datetime(ex_icustays_df['intime'], format="%Y-%m-%d %H:%M:%S")
    ex_icustays_df['outtime'] = pd.to_datetime(ex_icustays_df['outtime'], format="%Y-%m-%d %H:%M:%S")

    valid_lab['stay_id'] = np.nan

    for _, row in tqdm(valid_lab.iterrows(), total = valid_lab.shape[0], desc = 'assining_stay_id'):
        standard_icu = ex_icustays_df[ex_icustays_df['hadm_id'] == row.hadm_id].sort_values('intime')
            
        for _, stay in standard_icu.iterrows():
            
            if stay['intime'] - pd.Timedelta(days=valid_days) <= row['charttime'] <= stay['outtime']: # 몇 일 전 데이터까지 사용할 것인지?
                row['stay_id'] = stay['stay_id']
                new_lab_rows.append(row)
                break
            else:
                continue

    result = pd.DataFrame(new_lab_rows)
    
    return result


creat_df = assign_stayid_to_lab(labevents_df[labevents_df['itemid'] == 50912].reset_index(drop=True), ex_icustays_df, 7) # 중환자실 입원 전 7일 이내의 데이터만 사용.

creat_df['unit'] = creat_df['unit'].astype('string')
creat_df.to_parquet(ROOT_DIR/'preprocessed_yaib'/f'creat_for_labeling_df.parquet', index=False)

# admission weight
adm_weights = chartevents_df[chartevents_df['itemid'] == 226512].reset_index(drop = True)

# 사용 변수 설정
common_cols = ['stay_id', 'hadm_id', 'charttime', 'valuenum']

creat_df = creat_df[common_cols].sort_values(['stay_id', 'charttime']).reset_index(drop=True)
adm_weights = chartevents_df[chartevents_df['itemid'] == 226512][common_cols].reset_index(drop = True)

creat_df.rename(columns={'valuenum' : 'creatinine'}, inplace = True)
adm_weights.rename(columns={'valuenum' : 'admission_weight'}, inplace = True)
creat_df.rename(columns={'valuenum' : 'creatinine'}, inplace = True)


assining_stay_id: 100%|██████████| 925254/925254 [08:02<00:00, 1916.46it/s]


### 2-1-2) Caculate creatinine baseline & urine rate

In [292]:
# Exclusion Cohort용 baseline 계산

# 환자별로 우선 exclusion criteria를 위한 baseline 계산
def compute_baselines_for_excl(stay_df, creat_df):
    """
    YAIB 논문에 따르면 exclusion criteria를 계산하기 위해 입원 전 가장 최근값 or ICU 입원 후 가장 빠른 값이 4mg/dL이 넘으면 제외한다고 함.
    """
    
    def _baseline_preicu_last_or_earliest_in_icu(creat_df, icu_intime):
        # last prior to ICU
        prior = creat_df[creat_df['charttime'] < icu_intime]
        if not prior.empty:
            # 가장 최근값
            return prior.sort_values('charttime', ascending=False).iloc[0]['creatinine']
        
        # 없으면 ICU 내의 earliest
        in_icu = creat_df[creat_df['charttime'] >= icu_intime]

        if not in_icu.empty:
            return in_icu.sort_values('charttime').iloc[0]['creatinine']
        
        return np.nan
    
    rows = []
    for stay in tqdm(stay_df.stay_id.unique()):
        csub = creat_df[creat_df['stay_id'] == stay]
        icu_time = stay_df[stay_df['stay_id'] == stay]['intime'].values[0]
        
        bpre = _baseline_preicu_last_or_earliest_in_icu(csub, icu_time)
        rows.append({'stay_id':stay, 'baseline_preicu_or_earliest': bpre})

    return pd.DataFrame(rows)

# baseline 계산 
baselines_df = compute_baselines_for_excl(ex_icustays_df, creat_df)

# exclusion mask 생성
aki_exclude_mask = baselines_df['baseline_preicu_or_earliest'] > 4.0
aki_excluded_stays = baselines_df.loc[aki_exclude_mask, 'stay_id'].tolist()


100%|██████████| 62638/62638 [01:46<00:00, 587.82it/s]


In [338]:
# labeling 계산을 위한 baseline 계산

def compute_baseline_7d(group):
    """
    하나의 row마다 이전 7일 이내의 과거 값들 중 최소값을 baseline으로 설정
    """
    # set index -> rolling('7D') will consider previous 7 days up to current index
    g = group.set_index('charttime').sort_index()

    # rolling window of 7 days, exclude current row by closed='left'
    baseline_series = g['creatinine'].rolling('7D', closed='left').min()

    # baseline for earliest points will be NaN if no prior values
    g = g.assign(baseline_7d = baseline_series)

    return g.reset_index()

baseline_7d_df = creat_df.groupby('stay_id', group_keys = False).apply(compute_baseline_7d)
baseline_7d_df = pd.merge(baseline_7d_df, baselines_df, how = 'left', on = 'stay_id')
baseline_7d_df['baseline_7d'] = baseline_7d_df['baseline_7d'].fillna(baseline_7d_df['baseline_preicu_or_earliest']) # 만약 baseline이 결측값이면 icu 입원 전 마지막 or icu 입원 후 첫 번째 값으로 채움

C:\Users\korea\AppData\Local\Temp\ipykernel_27176\2357967660.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  baseline_7d_df = creat_df.groupby('stay_id', group_keys = False).apply(compute_baseline_7d)


In [346]:
# Baseline 기준으로 flag 계산
tqdm.pandas()

def compute_baseline_rules(group):
    """
    rule 1 : 해당 시점 기준 48시간 이내 0.3mg/dL 이상 증가 여부
    rule 2 : baseline 대비 1.5배 이상 증가 여부
    """
    # group: 하나의 stay_id 그룹
    group = group.copy()
    group.set_index('charttime', inplace=True)
    
    # 이전 row(즉, 현재 시점 제외)의 Creatinine 값을 기반으로 지난 48시간의 최소값을 계산
    group['min_creat_48h'] = group['creatinine'].shift(1).rolling('48h').min()
    group['rule1'] = (group['creatinine'] - group['min_creat_48h'] >= 0.3)
    group['rule1'] = group['rule1'].fillna(False)
    
    # Rule 2: 현재 Creatinine 값이 baseline_cr의 1.5배 이상인가?
    group['rule2'] = (group['creatinine'] >= group['baseline_7d'] * 1.5)
    
    return group.reset_index()

aki_baseline_flag = baseline_7d_df.groupby('stay_id', group_keys=False).progress_apply(compute_baseline_rules)
aki_baseline_flag['AKI_Annotation'] = np.where((aki_baseline_flag['rule1'] | aki_baseline_flag['rule2']),1,0)

100%|█████████▉| 62320/62394 [01:07<00:00, 963.16it/s]c:\Users\korea\miniconda3\envs\.venv\lib\site-packages\tqdm\std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 62394/62394 [01:11<00:00, 878.25it/s]


In [293]:
# urine rate 계산
def cal_urine_rate(outputevent_df, adm_weight_df):

    df = outputevent_df.groupby(['stay_id', 'charttime'], as_index=False)['value'].sum() # 먼저 summation을 취함

    df = df.sort_values(['stay_id','charttime'])
    df['prev_time'] = df.groupby('stay_id')['charttime'].shift(1)

    df['elapsed_hours'] = (df['charttime'] - df['prev_time']).dt.total_seconds() / 3600.0

    # apply rules: earliest -> 1h, cap at 24h, avoid zero
    df['elapsed_hours'] = df['elapsed_hours'].fillna(1.0)            # earliest = 1h
    df['elapsed_hours'] = df['elapsed_hours'].replace(0, 1.0)        # zero gap -> treat as 1h (or small eps)
    df['elapsed_hours'] = df['elapsed_hours'].clip(upper=24.0)       # max gap = 24h

    # 4) ml/hour 및 ml/kg/hour 계산
    df['ml_per_hour'] = df['value'] / df['elapsed_hours']

    # merge weight (use 75 kg if missing)
    df = df.merge(adm_weight_df[['stay_id','admission_weight']], on='stay_id', how='left')
    df['admission_weight'] = df['admission_weight'].fillna(75.0)

    df['urine_rate'] = df['ml_per_hour'] / df['admission_weight']

    return df

urine_rate_df = cal_urine_rate(outputevent_df=outputevents_df, adm_weight_df=adm_weights)
gc.collect()

23

In [350]:
# 이 urine rate을 기반으로 가중 평균을 취해서 KDIGO 기준 annotation flag를 생성
def add_timeweighted_uo(group, window_hours=6, min_coverage_ratio=0.8):
    g = group.set_index('charttime')

    # 1) interval별 volume 계산 (ml)
    g['volume_ml'] = g['urine_rate'] * g['admission_weight'] * g['elapsed_hours']

    # 2) 지난 6시간 동안 volume 합, 시간 합
    win = f'{window_hours}h'
    vol_roll = g['volume_ml'].rolling(win, min_periods=1)
    dt_roll  = g['elapsed_hours'].rolling(win, min_periods=1)

    g['vol_6h_sum']   = vol_roll.sum()
    g['hours_6h_sum'] = dt_roll.sum()

    # 3) 시간가중 평균 urine rate (mL/kg/h)
    #    UO_rate_6h = 총 volume / (weight * 실제 커버된 시간)
    g['uo_6h_timeweighted'] = (
        g['vol_6h_sum'] / (g['admission_weight'] * g['hours_6h_sum'])
    )

    # 4) KDIGO urine 기준: 6h 평균 < 0.5 mL/kg/h
    g['AKI_Annotation'] = (g['uo_6h_timeweighted'] < 0.5).astype(int)

    return g.reset_index()

aki_urine_flag = (
    urine_rate_df
    .groupby('stay_id', group_keys=False)
    .apply(add_timeweighted_uo)
)

C:\Users\korea\AppData\Local\Temp\ipykernel_27176\1640663980.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(add_timeweighted_uo)


### 2-1-3) AKI Annotation

In [360]:
# AKI Annotation
def add_aki_annotation(baseline_flag, urine_flag, stays):
    """
    사용해야 하는 변수 : weight, urine output, 입원 이전 creatinine(baseline 계산용), 입원 이후 creatinine
    """

    use_cols = ['stay_id', 'charttime', 'AKI_Annotation']
    res_aki = pd.concat([baseline_flag[use_cols], urine_flag[use_cols]], axis = 0)

    print(f'Drop Duplicates : {res_aki.shape[0]} --> ', end = ' ')
    res_aki = res_aki.drop_duplicates()
    
    print(f'Drop Duplicates : {res_aki.shape[0]}', end = '\n')

    res_aki = pd.merge(res_aki, stays[['stay_id', 'intime']], how = 'left', on = 'stay_id')

    return res_aki[res_aki['charttime'] >= res_aki['intime']][use_cols].reset_index(drop=True)

# Disease Annotation
aki_annot_df = add_aki_annotation(baseline_flag=aki_baseline_flag, urine_flag=aki_urine_flag, stays=ex_icustays_df)

gc.collect()

Drop Duplicates : 4416220 -->  Drop Duplicates : 4401943


0

In [363]:
# Free up for memory
del baseline_7d_df, baselines_df, urine_rate_df, adm_weights
gc.collect()

28

# 3) Extract Final dataframe

- 해야할 작업 : 같은 charttime의 var이면 평균내기, 각 환자마다 Intime 이후 24시간까지의 값만 활용, demographic 정보까지 통합하기 

In [504]:
def filter_until_24hours(df, stays, type):

    if type == 'output':
        use_cols = ['stay_id', 'charttime', 'var_name', 'value']
    else:
        use_cols = ['stay_id', 'charttime', 'var_name', 'valuenum']

    res = pd.merge(df[use_cols], stays[['stay_id', 'intime']], how = 'left', on = 'stay_id')
    res['max_time'] = res['intime'] + pd.Timedelta(days=1)
    res = res[res['charttime'] <= res['max_time']]

    if type == 'output':
        res = res.groupby(['stay_id', 'charttime', 'var_name'],as_index = False)['value'].sum()
        res = res.sort_values(['stay_id', 'charttime'])
        
    else:
        res = res.groupby(['stay_id', 'charttime', 'var_name'],as_index = False)['valuenum'].mean()
        res = res.sort_values(['stay_id', 'charttime'])
        res.rename(columns={'valuenum' : 'value'}, inplace = True)

    return res.reset_index(drop=True)

filtered_chartevents_df = filter_until_24hours(chartevents_df, ex_icustays_df, 'chart')
filtered_labevents_df = filter_until_24hours(valid_labevents, ex_icustays_df, 'lab')
filtered_outputevents_df = filter_until_24hours(outputevents_df, ex_icustays_df, 'output')


In [505]:
# static information
static_df = ex_icustays_df[['stay_id', 'anchor_age', 'gender']].melt(
    id_vars = ['stay_id'],
    value_vars = ['anchor_age', 'gender'],
    var_name = 'var_name',
    value_name = 'value'
)

wh_df = filtered_chartevents_df[filtered_chartevents_df['var_name'].isin(['weight', 'height'])].groupby(['stay_id', 'var_name'],as_index = False).mean()[['stay_id', 'var_name', 'value']]

static_df = pd.concat([static_df, wh_df], axis = 0).reset_index(drop=True)

del wh_df

In [506]:
# Final 합치기 

filtered_chartevents_df = filtered_chartevents_df[~filtered_chartevents_df['var_name'].isin(['weight', 'height'])] # height, weight 제거

dynamics_df = pd.concat([filtered_chartevents_df, filtered_labevents_df, filtered_outputevents_df], axis = 0).sort_values(['stay_id','charttime']).reset_index(drop=True)

del filtered_chartevents_df, filtered_labevents_df, filtered_outputevents_df

# 4) Mortality, LoS, Disease Prediction Labeling

In [ ]:
# Reference : An Extensive Data Preprocessing Pipeline for MIMIC-IV

def add_inunit_mortality_to_icustays(stays):

    df = stays.copy()
    mortality = df.dod.notnull() & ((df.intime <= df.dod) & (df.outtime >= df.dod))
    mortality = mortality | (df.deathtime.notnull() & ((df.intime <= df.deathtime) & (df.outtime >= df.deathtime)))
    df['mortality_inunit'] = mortality.astype(int)
    
    return df

def add_los_to_icustays(stays): 
    """
    Regerssion Task로 설정 max는 7일
    """

    df = stays.copy()
    df['los_reg'] = ((df['los'] - 1) * 24).astype(int) 
    df['los_reg'] = df['los_reg'].clip(upper=168)
    
    return df

# 규민이형 말씀 : AKI를 training 할 때는 입원 후 하루 기간동안 AKI가 왔어도 사용한 경우가 있으나, 평가할 때는 빼는 게 맞는 거 같다는 생각.
def prediction_labeling(df : pd.DataFrame, annot_df : pd.DataFrame, prediction_hour : int = 8,
                        label_col_name : str = 'AKI_next_6h', annot_col_name : str = 'AKI_Annotation'):
    
    def _labeling(x):

        labels = [
            1 if x[annot_col_name].iloc[idx+1:idx+1+prediction_window].sum() >= 1 else 0
            for idx in range(len(x))
        ]
        x[label_col_name] = labels

        return x

    df['intime_next_24h'] = df['intime'] + pd.Timedelta(days=1)

    annot_df[[]]
    labeled_df = pd.merge(df, annot_df, how = 'left', on = ['stay_id', 'time_since_ICU'])
    labeled_df[annot_col_name] = labeled_df[annot_col_name].fillna(0)

    # generates sofa_next12h values within each stay, ensuring that predictions don’t span across different stays
    # The sofa_next12h column contains 1 if a future high SOFA score is anticipated and 0 otherwise, providing an actionable label for predictive modeling
    labeled_df = labeled_df.groupby('stay_id', group_keys=False).progress_apply(_labeling)

    return labeled_df

In [ ]:
stays = pd.merge(ex_icustays_df, admissions_df[['subject_id', 'hadm_id', 'deathtime', 'hospital_expire_flag', 'admittime', 'dischtime']], how='left', on = ['subject_id', 'hadm_id'])

for col in ['deathtime', 'admittime', 'dischtime', 'intime', 'outtime']:
    stays[col] = pd.to_datetime(stays[col], format="%Y-%m-%d %H:%M:%S")

outcome_df = add_inunit_mortality_to_icustays(stays)
outcome_df = add_los_to_icustays(outcome_df)

# # Diesease Early Prediction task
## 이 AKI를 어떻게 라벨링 할지 코드 다시 짜야함.
preidiction_hour = 6 # 6h
outcome_df = prediction_labeling(outcome_df, aki_annot_df, preidiction_hour, 'AKI_next_6h', 'AKI_Annotation') 

In [463]:
# AKI 라벨링 시 Cohort를 다르게 할지 아니면 그대로 할지 이런 것들도 고민해봐야 함.
aki_annot_df

,stay_id,charttime,AKI_Annotation
0,30000153,2174-09-29 15:37:00,0
1,30000153,2174-09-30 03:34:00,0
2,30000213,2162-06-21 14:02:00,0
3,30000213,2162-06-21 20:43:00,0
4,30000213,2162-06-22 04:00:00,0
...,...,...,...
4308236,39999858,2167-04-29 11:00:00,0
4308237,39999858,2167-04-29 12:00:00,0
4308238,39999858,2167-04-29 13:00:00,0
4308239,39999858,2167-04-29 15:00:00,0


In [518]:
dynamics_df.groupby('stay_id')['charttime'].count().describe()

count    62638.000000
mean       251.831492
std         75.770714
min          5.000000
25%        204.000000
50%        237.000000
75%        287.000000
max       2033.000000
Name: charttime, dtype: float64

In [494]:
# 저장할 데이터프레임들
static_df
dynamics_df
outcome_df
aki_annot_df
aki_excluded_stays

[34100191,
 32359580,
 35265231,
 32119961,
 34146568,
 30374965,
 37293400,
 39492446,
 33083787,
 34961868,
 38574326,
 31142025,
 36535245,
 36472818,
 38773518,
 31975834,
 38942593,
 30879445,
 32557628,
 38335930,
 34105920,
 34751092,
 36576905,
 32813762,
 32579510,
 37885860,
 34428987,
 30484216,
 36754593,
 39658632,
 33544024,
 31983721,
 38213665,
 39307906,
 33287202,
 31093528,
 34921121,
 37065927,
 35202273,
 33474714,
 30129595,
 39722652,
 35988303,
 35257230,
 35459203,
 39914934,
 30752201,
 34776840,
 31578020,
 31569899,
 30472744,
 35006975,
 31179345,
 35186859,
 32735632,
 33352159,
 30747248,
 33712680,
 38562441,
 37542315,
 31219818,
 37337185,
 33780803,
 38853861,
 38972779,
 36139734,
 31433506,
 33413600,
 38490796,
 34699095,
 37039117,
 39249300,
 34241430,
 37663114,
 30924879,
 32205206,
 39847905,
 33485454,
 33270365,
 33505376,
 30392749,
 31020687,
 31091474,
 38416275,
 36232907,
 38069518,
 33650320,
 39428217,
 32259031,
 39703863,
 38461758,

# 5) Final DataFrame

In [ ]:
# MIMIC에서는 기존의 charttime 형태 -> eICU처럼 입원 후 몇 분에 대한 timestamp인지를 나타내도록 변경
def charttime_to_min(df, stays):
    res = pd.merge(df, stays[['stay_id', 'intime']], how = 'left', on ='stay_id')

    res['charttime'] = ((res['charttime'] - res['intime']).dt.total_seconds() / 60).apply(np.ceil).astype(int)

    return res.drop(columns = 'intime')

dynamics_df = charttime_to_min(dynamics_df, ex_icustays_df)
aki_annot_df = charttime_to_min(aki_annot_df, ex_icustays_df)

In [ ]:
# Export Data
dynamics_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip', index=False)
static_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'static_df.csv.gz', compression='gzip', index=False)
outcome_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'outcome_df.csv.gz', compression='gzip', index=False)
aki_annot_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'aki_annot_df.csv.gz', compression='gzip', index=False)
np.save(ROOT_DIR/'preprocessed_yaib'/'aki_exclude_stayid.npy', np.array(aki_excluded_stays))

# 6) Analysis of events

In [3]:
ROOT_DIR = '/Users/korea/EHRTTA/data/miiv'
ROOT_DIR = Path(ROOT_DIR)

# dynamics_df = pd.read_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip')
static_df = pd.read_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip')

# print(dynamics_df.var_name.nunique())
print(static_df.var_name.nunique())


4


In [ ]:
# static data의 anchor age, gender 변경
mapping = {'anchor_age' : 'age', 'gender' : 'sex'}
static_df['var_name'] = static_df['var_name'].replace(mapping)

static_df.to_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip', index = False)

In [6]:
lab_list = MAPPING_DF[~MAPPING_DF['category'].isin(['respiratory', 'vitals', 'output', 'demographics'])].var_name.unique()
vital_list = MAPPING_DF[MAPPING_DF['category'].isin(['respiratory', 'vitals'])].var_name.unique()
output_list = MAPPING_DF[MAPPING_DF['category'].isin(['output'])].var_name.unique()

lab = dynamics_df[dynamics_df['var_name'].isin(lab_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'lab'})
vitals = dynamics_df[dynamics_df['var_name'].isin(vital_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'vitals'})
output = dynamics_df[dynamics_df['var_name'].isin(output_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'output'})

analysis_df = pd.merge(lab, vitals, how='left', on ='stay_id').merge(output, how='left', on='stay_id')

analysis_df['total'] = analysis_df['lab'] + analysis_df['vitals'] + analysis_df['output']

for col in analysis_df.columns:
    if col != 'stay_id':
        print(f'----------------Number of {col} events----------------')
        display(analysis_df[col].describe())

----------------Number of lab events----------------


count    62169.000000
mean        62.487687
std         38.372770
min          1.000000
25%         34.000000
50%         54.000000
75%         83.000000
max        409.000000
Name: lab, dtype: float64

----------------Number of vitals events----------------


count    62124.000000
mean       177.180977
std         50.629749
min          1.000000
25%        153.000000
50%        163.000000
75%        190.000000
max       1872.000000
Name: vitals, dtype: float64

----------------Number of output events----------------


count    59757.000000
mean        13.688606
std          6.944784
min          1.000000
25%          7.000000
50%         15.000000
75%         20.000000
max         36.000000
Name: output, dtype: float64

----------------Number of total events----------------


count    59757.000000
mean       254.312097
std         74.773504
min         13.000000
25%        206.000000
50%        239.000000
75%        289.000000
max       2033.000000
Name: total, dtype: float64